In [1]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [2]:
import csv, json, os, math, numbers, pandas, re, scipy, scipy.sparse, shutil
import subprocess, sys, threading, time, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [3]:
pandas.options.display.max_colwidth = 300

# Try to connect

In [6]:
google_sheets = 'https://docs.google.com/spreadsheets/d/1eP7P0drQ69KTcumHHoZMoIqgp0g-OBVn26pCvcUfs_U/edit?usp=sharing'
# Generated csv path from sheets URL using http://localhost:8815/notebooks/uwsgi/dotmaptiles/data-visualization-tools/examples/lodes/sheetUrl2CsvUrl.ipynb
sec8_csv_path = 'https://docs-proxy.cmucreatelab.org/spreadsheets/d/1eP7P0drQ69KTcumHHoZMoIqgp0g-OBVn26pCvcUfs_U/export?format=csv&gid=1067479687'

In [8]:
sec8_data = pandas.read_csv(sec8_csv_path)
sec8_data

,Census Block Group,Voucher Total
0,420030103003,1
1,420030103004,56
2,420030201001,59
3,420030201002,40
4,420030203001,2
5,420030305001,86
6,420030305002,102
7,420030402001,18
8,420030405002,1
9,420030406001,4


### Read 2010 block geoids and 2010 block populations

In [9]:
block_populations = numpy.load('columncache/census2010_block2010/p001001.numpy')
print 'block_populations has', sum(block_populations), 'total people'

block_populations has 308745538 total people


In [10]:
# block_geoids_2010 = [row[0] for row in query_psql("SELECT geoid2010 FROM sf1_2010_block_p001 order by blockidx2010")]
block_geoids_2010 = json.load(open('block_geoids_2010.json'))
print 'There are', len(block_geoids_2010), 'blocks'

assert(len(block_geoids_2010) + 1 == len(block_populations))

There are 11078297 blocks


### Compute 2010 population by tract and block indices from tract

In [14]:
blockgroup_populations = {}
blockgroup_block_indexes = {}

for block_index_minus_one, block_geoid in enumerate(block_geoids_2010):
    block_index = block_index_minus_one + 1
    blockgroup_name = block_geoid[0:12] # SSCCCTTTTTTG
    if blockgroup_name not in blockgroup_populations:
        blockgroup_populations[blockgroup_name] = 0
        blockgroup_block_indexes[blockgroup_name] = []
    blockgroup_populations[blockgroup_name] += block_populations[block_index]
    blockgroup_block_indexes[blockgroup_name].append(block_index)

print 'There are', len(blockgroup_populations), 'blockgroups'
print 'blockgroup_populations has', sum(blockgroup_populations.values()), 'people'

There are 217740 blockgroups
blockgroup_populations has 308745538 people


### Interpolate and write columns for data file

In [16]:
output_col = numpy.zeros(block_populations.size, dtype=numpy.float32)
len(output_col)

11078298

In [34]:
!mkdir columncache/pa_pittsburgh_sec8

In [35]:
for r in range(len(sec8_data)):
    blockgroup = str(sec8_data['Census Block Group'][r])
    input_pop = sec8_data['Voucher Total'][r]
    print '%s has %d people' % (blockgroup, input_pop)
    for block_index in blockgroup_block_indexes[blockgroup]:
        if block_populations[block_index]:
            interp = input_pop * float(block_populations[block_index]) / blockgroup_populations[blockgroup]
            output_col[block_index] = interp 
            print '  assigning %s with %.2f people' % (block_geoids_2010[block_index - 1], interp)

output_col_path = 'columncache/pa_pittsburgh_sec8/sec8_vouchers_20180326.float32'
output_col.tofile(output_col_path + '.tmp')
os.rename(output_col_path + '.tmp', output_col_path)
sys.stdout.write('Created %s with sum %f\n' % (output_col_path, output_col.sum()))


420030103003 has 1 people
  assigning 420030103003001 with 0.00 people
  assigning 420030103003002 with 0.00 people
  assigning 420030103003003 with 0.09 people
  assigning 420030103003006 with 0.04 people
  assigning 420030103003007 with 0.09 people
  assigning 420030103003008 with 0.11 people
  assigning 420030103003009 with 0.07 people
  assigning 420030103003010 with 0.03 people
  assigning 420030103003012 with 0.10 people
  assigning 420030103003015 with 0.14 people
  assigning 420030103003016 with 0.12 people
  assigning 420030103003017 with 0.16 people
  assigning 420030103003018 with 0.05 people
420030103004 has 56 people
  assigning 420030103004002 with 0.35 people
  assigning 420030103004003 with 14.49 people
  assigning 420030103004004 with 3.18 people
  assigning 420030103004007 with 4.42 people
  assigning 420030103004008 with 3.71 people
  assigning 420030103004009 with 4.77 people
  assigning 420030103004010 with 0.18 people
  assigning 420030103004011 with 10.07 people


  assigning 420031011002014 with 0.06 people
  assigning 420031011002018 with 0.07 people
  assigning 420031011002019 with 0.21 people
  assigning 420031011002020 with 1.12 people
  assigning 420031011002021 with 0.15 people
  assigning 420031011002022 with 0.13 people
  assigning 420031011002023 with 0.47 people
  assigning 420031011002024 with 0.95 people
  assigning 420031011002025 with 0.25 people
  assigning 420031011002026 with 0.48 people
  assigning 420031011002027 with 1.27 people
  assigning 420031011002028 with 0.07 people
  assigning 420031011002030 with 0.13 people
420031011003 has 3 people
  assigning 420031011003000 with 0.01 people
  assigning 420031011003002 with 0.04 people
  assigning 420031011003003 with 0.24 people
  assigning 420031011003004 with 0.24 people
  assigning 420031011003006 with 0.18 people
  assigning 420031011003007 with 0.13 people
  assigning 420031011003008 with 0.08 people
  assigning 420031011003009 with 0.25 people
  assigning 420031011003010 w

  assigning 420031413003000 with 5.54 people
  assigning 420031413003001 with 0.39 people
  assigning 420031413003002 with 4.63 people
  assigning 420031413003003 with 2.17 people
  assigning 420031413003004 with 1.27 people
420031414003 has 1 people
  assigning 420031414003000 with 0.03 people
  assigning 420031414003001 with 0.04 people
  assigning 420031414003003 with 0.05 people
  assigning 420031414003005 with 0.19 people
  assigning 420031414003006 with 0.11 people
  assigning 420031414003007 with 0.00 people
  assigning 420031414003008 with 0.02 people
  assigning 420031414003010 with 0.01 people
  assigning 420031414003011 with 0.02 people
  assigning 420031414003013 with 0.03 people
  assigning 420031414003014 with 0.03 people
  assigning 420031414003015 with 0.03 people
  assigning 420031414003016 with 0.02 people
  assigning 420031414003017 with 0.01 people
  assigning 420031414003019 with 0.02 people
  assigning 420031414003020 with 0.01 people
  assigning 420031414003021 w

  assigning 420031920005006 with 0.41 people
  assigning 420031920005007 with 1.07 people
  assigning 420031920005008 with 0.35 people
  assigning 420031920005009 with 0.07 people
  assigning 420031920005010 with 0.15 people
420032022001 has 35 people
  assigning 420032022001000 with 0.18 people
  assigning 420032022001001 with 0.42 people
  assigning 420032022001002 with 0.84 people
  assigning 420032022001003 with 1.70 people
  assigning 420032022001004 with 0.60 people
  assigning 420032022001005 with 2.81 people
  assigning 420032022001006 with 0.21 people
  assigning 420032022001007 with 0.05 people
  assigning 420032022001008 with 1.65 people
  assigning 420032022001009 with 0.37 people
  assigning 420032022001010 with 0.37 people
  assigning 420032022001011 with 0.79 people
  assigning 420032022001012 with 0.10 people
  assigning 420032022001013 with 1.44 people
  assigning 420032022001014 with 0.58 people
  assigning 420032022001015 with 0.94 people
  assigning 420032022001016 

  assigning 420033103001003 with 0.03 people
  assigning 420033103001004 with 0.03 people
  assigning 420033103001005 with 0.00 people
  assigning 420033103001006 with 0.02 people
  assigning 420033103001008 with 0.00 people
  assigning 420033103001010 with 0.03 people
  assigning 420033103001011 with 0.03 people
  assigning 420033103001012 with 0.02 people
  assigning 420033103001014 with 0.03 people
  assigning 420033103001015 with 0.03 people
  assigning 420033103001016 with 0.05 people
  assigning 420033103001017 with 0.05 people
  assigning 420033103001019 with 0.04 people
  assigning 420033103001020 with 0.02 people
  assigning 420033103001021 with 0.03 people
  assigning 420033103001025 with 0.01 people
  assigning 420033103001028 with 0.02 people
  assigning 420033103001029 with 0.01 people
  assigning 420033103001030 with 0.01 people
  assigning 420033103001046 with 0.01 people
  assigning 420033103001048 with 0.01 people
  assigning 420033103001052 with 0.00 people
  assignin

In [36]:
!ls -lR columncache/pa*

columncache/pa_allegheny_county_evictions:
total 389484
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:36 evictions2008.float32
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:36 evictions2009.float32
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:36 evictions2010.float32
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:36 evictions2011.float32
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:36 evictions2012.float32
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:36 evictions2013.float32
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:37 evictions2014.float32
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:37 evictions2015.float32
-rw-r--r-- 1 rsargent rsargent 44313188 Nov 18 13:37 evictions2016.float32

columncache/pa_pittsburgh_sec8:
total 43276
-rw-rw-r-- 1 rsargent rsargent 44313192 May  1 10:41 sec8_vouchers_20180326.float32


In [41]:
!ls -l columncache/census2010_block2010/*float32 | head

-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0050002.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0050003.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0050004.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0050005.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0050006.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0050007.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0050008.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0110002.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census2010_block2010/H0110003.float32
-rw-r--r-- 1 rsargent rsargent 44313192 Aug 19  2017 columncache/census20

In [38]:
22156676*2

44313352